In [9]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import math

Using TensorFlow backend.


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cpu':
    print('cpu')
else:
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0))

GeForce GTX 1050 Ti with Max-Q Design


In [11]:
import pandas as pd

texts = pd.read_csv('D:/Project/Work_Project/final/data/processed/Dataset_text_1.csv', encoding='utf8')

In [12]:
texts = texts.dropna(axis='index', how='any', subset=['CLASS'])

In [13]:
mir_texts = texts[texts['CLASS'] == 0]
maf_texts = texts[texts['CLASS'] == 1]

In [14]:
sentences = np.concatenate([maf_texts['TEXT'].values, mir_texts['TEXT'].values])

sentences = ['[CLS] ' + str(sentence) + ' [SEP]' for sentence in sentences]
labels = [[1] for _ in range(maf_texts.shape[0])] + [[0] for _ in range(mir_texts.shape[0])]

In [15]:
assert len(sentences) == len(labels) == maf_texts.shape[0] + mir_texts.shape[0]

In [16]:
print(sentences[100])

[CLS] Всем привет ещё раз кто был со вчерашнего дня Кто кто сегодня впервые Ну что вчера было весело было здорово Мафия унижала нас моральное интеллектуальное не знаю Надеюсь сегодня это традиция Ну вот я желаю всем хорошей игры я не знаю как вообще можно по первому кругу когото подозревать не подозревать я не знаю это ведущий ведущий показывает какието жесты возле головы по поводу речей Я не знаю мне не понравилось ни одна речь Но типа я думаю что посмотреть что будет дальше всем удачи всем хорошей игры класс Спасибо [SEP]


In [17]:
from sklearn.model_selection import train_test_split

train_sentences, test_sentences, train_gt, test_gt = train_test_split(sentences, labels, test_size=0.3)

In [18]:
print(len(train_sentences), len(test_sentences))

348 150


In [19]:
from pytorch_transformers import BertTokenizer, BertConfig


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

tokenized_texts = [tokenizer.tokenize(sent) for sent in train_sentences]
print (tokenized_texts[0])

['[CLS]', 'н', '##е', 'а', '##л', '##е', '##к', '##с', '##е', '##и', 'mafia', '##ua', 'о', '##л', '##ь', '##г', '##а', 'п', '##о', '##т', '##о', '##м', '##у', 'ч', '##т', '##о', 'к', '##о', '##г', '##д', '##а', 'н', '##а', '##ч', '##а', '##л', '##а', 'у', '##т', '##о', '##ч', '##н', '##я', '##т', '##ь', 'п', '##р', '##о', 'в', '##с', '##я', '##к', '##и', '##е', 'э', '##т', '##и', 'о', '##р', '##г', '##ан', '##и', '##з', '##а', '##ц', '##и', '##о', '##н', '##н', '##ы', '##е', 'н', '##ю', '##ан', '##с', '##ы', 'ф', '##р', '##е', '##г', '##а', '##т', 'з', '##а', '##м', '##е', '##т', '##и', '##л', 'э', '##т', '##у', 'с', '##и', '##т', '##у', '##а', '##ц', '##и', '##ю', 'в', '##о', '##т', 'и', 'в', '##о', '##з', '##м', '##о', '##ж', '##н', '##о', 'м', '##а', '##с', '##я', 'р', '##е', '##ш', '##и', '##л', '##а', 'у', '##с', '##т', '##р', '##ан', '##и', '##т', '##ь', 'в', '##о', '##т', 'п', '##о', '##э', '##т', '##о', '##м', '##у', 'п', '##о', '##ка', 'ч', '##т', '##о', 'м', '##о', '##и', 'п'

In [20]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(
    input_ids,
    maxlen=100,
    dtype="long",
    truncating="post",
    padding="post"
)
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

Token indices sequence length is longer than the specified maximum sequence length for this model (604 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (591 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1064 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

In [21]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids, train_gt, 
    random_state=42,
    test_size=0.1
)

train_masks, validation_masks, _, _ = train_test_split(
    attention_masks,
    input_ids,
    random_state=42,
    test_size=0.1
)

In [22]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

In [23]:
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [24]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(
    train_data,
    sampler=RandomSampler(train_data),
    batch_size=32
)

In [25]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(
    validation_data,
    sampler=SequentialSampler(validation_data),
    batch_size=32
)

In [26]:
from pytorch_transformers import AdamW, BertForSequenceClassification

In [27]:
from pytorch_transformers import BertForQuestionAnswering, BertForTokenClassification

In [28]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [29]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5)



In [30]:
from IPython.display import clear_output

# Будем сохранять loss во время обучения
# и рисовать график в режиме реального времени
train_loss_set = []
train_loss = 0


# Обучение
# Переводим модель в training mode
model.train()

for _ in range(5):
  for step, batch in enumerate(train_dataloader):
      # добавляем батч для вычисления на GPU
    #   batch = tuple(t.to(device) for t in batch)
      # Распаковываем данные из dataloader
      b_input_ids, b_input_mask, b_labels = batch
      
      # если не сделать .zero_grad(), градиенты будут накапливаться
      optimizer.zero_grad()
      
      # Forward pass
      loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

      train_loss_set.append(loss[0].item())  
      
      # Backward pass
      loss[0].backward()
      
      # Обновляем параметры и делаем шаг используя посчитанные градиенты
      optimizer.step()

      # Обновляем loss
      train_loss += loss[0].item()
      
      # Рисуем график
      clear_output(True)
      plt.plot(train_loss_set)
      plt.title("Training loss")
      plt.xlabel("Batch")
      plt.ylabel("Loss")
      plt.show()
      
print("Loss на обучающей выборке: {0:.5f}".format(train_loss / len(train_dataloader)))


# Валидация
# Переводим модель в evaluation mode
model.eval()

valid_preds, valid_labels = [], []

for batch in validation_dataloader:   
    # добавляем батч для вычисления на GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Распаковываем данные из dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # При использовании .no_grad() модель не будет считать и хранить градиенты.
    # Это ускорит процесс предсказания меток для валидационных данных.
    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Перемещаем logits и метки классов на CPU для дальнейшей работы
    logits = logits[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    batch_preds = np.argmax(logits, axis=1)
    batch_labels = np.concatenate(label_ids)     
    valid_preds.extend(batch_preds)
    valid_labels.extend(batch_labels)

print("Процент правильных предсказаний на валидационной выборке: {0:.2f}%".format(
    accuracy_score(valid_labels, valid_preds) * 100
))

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.IntTensor instead (while checking arguments for embedding)